### Correlation Based Recommender - with Item Based Similarity 

- Based on Pearson correlation 
- This is based on Item based similarity, recommend based on item rating .. basically recommending an Item based on something that the user already rated.



In [55]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

###  Loading Data and setting up for analysis

In [6]:
ratings = pd.read_csv('rating_final.csv')
cuisine = pd.read_csv('chefmozcuisine.csv')
geodata = pd.read_csv('geoplaces2.csv',encoding='latin-1')
places = geodata[['placeID','name']]

In [44]:
ratings.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [35]:
rating_data = pd.DataFrame(ratings.groupby('placeID')['rating'].mean())
rating_data['rating_count'] = ratings.groupby('placeID')['rating'].count()
rating_data.sort_values('rating_count',ascending=False).head()

,rating,rating_count
placeID,,
135085,1.333333,36
132825,1.281250,32
135032,1.178571,28
135052,1.280000,25
132834,1.000000,25


In [42]:
places[places['placeID']==135085]

,placeID,name
121,135085,Tortas Locas Hipocampo


In [43]:
cuisine[cuisine['placeID']==135085]

,placeID,Rcuisine
44,135085,Fast_Food


### Preparing Data for Analysis 

In [48]:
places_crosstab = pd.pivot_table(data=ratings,values = 'rating', index='userID',columns='placeID')
places_crosstab.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
U1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
U1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# taking the most popular restraunt 
tortas_rating = places_crosstab[135085]
tortas_rating[tortas_rating>=0]

userID
U1001    0.0
U1002    1.0
U1007    1.0
U1013    1.0
U1016    2.0
U1027    1.0
U1029    1.0
U1032    1.0
U1033    2.0
U1036    2.0
U1045    2.0
U1046    1.0
U1049    0.0
U1056    2.0
U1059    2.0
U1062    0.0
U1077    2.0
U1081    1.0
U1084    2.0
U1086    2.0
U1089    1.0
U1090    2.0
U1092    0.0
U1098    1.0
U1104    2.0
U1106    2.0
U1108    1.0
U1109    2.0
U1113    1.0
U1116    2.0
U1120    0.0
U1122    2.0
U1132    2.0
U1134    2.0
U1135    0.0
U1137    2.0
Name: 135085, dtype: float64

###  Finding similarity with Correlation and Evaluating 

So tortas is the most popular and rated restuarant hence, we will try to evaluate and look at similar to tortas kind of resturants. 

In [91]:
similar_to_tortas = places_crosstab.corrwith(tortas_rating)
similar_to_tortas.head()

placeID
132560         NaN
132561         NaN
132564         NaN
132572   -0.428571
132583         NaN
dtype: float64

In [66]:
corr_tab = pd.DataFrame(similar_to_tortas,columns=['PearsonR'])
corr_tab.dropna(inplace=True)
corr_tab.head()

,PearsonR
placeID,
132572,-0.428571
132723,0.301511
132754,0.930261
132825,0.700745
132834,0.814823


In [81]:
total_corr_summary = corr_tab.join(rating_data['rating_count'])
total_corr_summary[total_corr_summary['rating_count']>=10].sort_values('PearsonR',ascending=False).head(10)

,PearsonR,rating_count
placeID,,
135076,1.000000,13
135085,1.000000,36
135066,1.000000,12
132754,0.930261,13
135045,0.912871,13
135062,0.898933,21
135028,0.892218,15
135042,0.881409,20
135046,0.867722,11


In [87]:
similar_to_tortas = pd.DataFrame([132754,135045,135062,135028,135042,135046,132872],index=np.arange(7),columns=['placeID'])
summary = pd.merge(similar_to_tortas,cuisine,on='placeID')

In [88]:
summary

,placeID,Rcuisine
0,132754,Mexican
1,135028,Mexican
2,135042,Chinese
3,135046,Fast_Food
4,132872,American


In [89]:
places[places['placeID']==135046]

,placeID,name
42,135046,Restaurante El Reyecito


So, we will recommend Restaurante El Reyecito !! 

What we did .. we looked at the top places with correlation and populatrity. Out of the top 5-7 we looked at another resturant that servers Fast Food.